# Data Preparation

The metrics that will be used to evaluate this stage are:

- **Integration**, this is, converting different data value formats AND entity matching between different sources;

- **Quality**, assessment of dimensions;

- **Cleaning**, systematic redundancy removal of redundant attributes, missing data, replace MVs with complex methods (e.g. regression, classification) with correct experimental setup, identify and discuss outliers and address them with complex approaches (technical or domain-dependent);

- **Transformation** for algorithm compatibility, adequate complex discretization or rescaling;

- **Feature Engineering and Selection** from tabular data, complex methods (e.g. aggregation) and knowledge (e.g. business concepts), and correct and combined use of filter and wrapper based methods;

- **Sampling** for domain-specific purposes, focus on the appropriate subset of the population, and for development, start with a very small sample and scale up to a significant sample;

- **Unbalanced** - you used advanced methods (e.g. SMOTE) correctly

## 0 - Load data and define functions

In [115]:
import pandas as pd
import os

awards_players = pd.read_csv("data/awards_players.csv")
coaches = pd.read_csv("data/coaches.csv")
players_teams = pd.read_csv("data/players_teams.csv")
players = pd.read_csv("data/players.csv")
series_post = pd.read_csv("data/series_post.csv")
teams_post = pd.read_csv("data/teams_post.csv")
teams = pd.read_csv("data/teams.csv")

tables = {
    "Awards Players": awards_players,
    "Coaches": coaches,
    "Players Teams": players_teams,
    "Players": players,
    "Series Post": series_post,
    "Teams Post": teams_post,
    "Teams": teams
}

In [116]:
# remove columns where all the entries have the same value
def clean_remove_columns_equal(name):
    
    before = len(tables[name].columns)
    nunique = tables[name].nunique()
    cols_to_drop = nunique[nunique == 1].index
    tables[name] = tables[name].drop(cols_to_drop, axis=1)
    
    if( before != len( tables[name].columns)):
        print( f"(Only one value) {name} went from {before} to {len(tables[name].columns)} columns : {cols_to_drop.to_list()}")

# remove columns where all the entries have null values
def clean_remove_columns_na(name):
    
    before_cols = tables[name].columns
    tables[name] = tables[name].dropna(axis=1, how='all')
    after_cols = tables[name].columns
    
    if(len(before_cols) != len(after_cols)):
        print(f"(Only missing data) {name} went from {len(before_cols)} to {len(after_cols)} columns : {list(filter(lambda x: x not in after_cols, before_cols))}")

# identify columns that have null values
def clean_identify_columns_na(name):
    
    cols_with_empty_values = tables[name].columns[tables[name].isnull().any() ].to_list()
    size = len(cols_with_empty_values)
    
    if size > 0:
        print(f"{name} has {size} columns with missing data : {cols_with_empty_values}")
        
# identify pairs of very correlated features
def clean_identify_correlated_features(name):    
    features = []
    data_converted = tables[name].copy()
    
    for col in data_converted.select_dtypes(include=['object']).columns:
        data_converted[col], _ = pd.factorize(data_converted[col])

    matrix = data_converted.corr()
    for i in range(len(matrix.columns)):
        for j in range(i):
            value = matrix.iloc[i,j]
            if abs(value) > 0.95:
                name_i = matrix.columns[i]
                name_j = matrix.columns[j]
                features.append((name_i,name_j,value))

    return features

# write contents to file
def table_to_csv(name):
    os.makedirs('data_prepared', exist_ok=True)
    path = os.path.join('data_prepared', name.lower().replace(' ', '_') + ".csv")
    tables[name].to_csv(path, index=False)

## 1 - Clean all the datasets

We start by removing columns that have the same value in every entry and columns that have all entries empty.

In [117]:
# clean datasets
for table in tables.keys():
    clean_remove_columns_equal(table)
    clean_remove_columns_na(table)

(Only one value) Awards Players went from 4 to 3 columns : ['lgID']
(Only one value) Coaches went from 9 to 8 columns : ['lgID']
(Only one value) Players Teams went from 43 to 42 columns : ['lgID']
(Only one value) Players went from 10 to 8 columns : ['firstseason', 'lastseason']
(Only one value) Series Post went from 9 to 7 columns : ['lgIDWinner', 'lgIDLoser']
(Only one value) Teams Post went from 5 to 4 columns : ['lgID']
(Only one value) Teams went from 61 to 53 columns : ['lgID', 'seeded', 'tmORB', 'tmDRB', 'tmTRB', 'opptmORB', 'opptmDRB', 'opptmTRB']
(Only missing data) Teams went from 53 to 52 columns : ['divID']


We then removed columns that do not contain relevant information.

In [118]:
# tables['Players'] = tables['Players'].drop(['college', 'collegeOther', 'birthDate', 'deathDate'], axis=1)

We then need to change the `tmID` column of all the datasets as it may not correspond to the `franchID` column of the teams dataset.

In [119]:
mapTeam = {}
for index, row in teams[['tmID', 'franchID']].iterrows():
    mapTeam [row['tmID']] = row['franchID']

tables['Teams']['tmID'] = tables['Teams']['tmID'].replace(mapTeam) 
tables['Coaches']['tmID'] = tables['Coaches']['tmID'].replace(mapTeam) 
tables['Players Teams']['tmID'] = tables['Players Teams']['tmID'].replace(mapTeam) 
tables['Series Post']['tmIDWinner'] = tables['Series Post']['tmIDWinner'].replace(mapTeam) 
tables['Series Post']['tmIDLoser'] = tables['Series Post']['tmIDLoser'].replace(mapTeam) 
tables['Teams Post']['tmID'] = tables['Teams Post']['tmID'].replace(mapTeam)

## 2. Prepare teams.csv

There are several attributes that are highly correlated with each other and may not be necessary in the Teams dataset. So let's make some changes:

- As we had already noticed previously, `tmID` and `franchID` are always the same, except in some specific cases. They are not equal only when the name under which the team participated in the competition (`tmID`) is not the same as its current name (`franchID`), that is, it changed its name.

- The `name` and `arena` attributes, which give us the full name of the team and the name of its arena, are not necessary, as `tmID` is sufficient as an identifier.

- Regarding the attributes referring to performance in the season, knowing that the `GP` attribute varies between two values, 32 and 34, depending on the year. In both cases the number of games is even, with half of them played at home and the rest away. Therefore:

    - The `GP` attribute can be obtained by doing `won + lost`.
    
    - The `homeL` attribute can be obtained by doing `GP/2 - homeW`.
    
    - The `awayL` attribute can be obtained by doing `lost - homeL`.
    
    - The `awayW` attribute can be obtained by doing `won - homeW`.

- In relation to the attributes referring to offensive statistics, the same ideas can be replicated for defensive statistics.

    - The attribute `o_reb` can be obtained `o_oreb + o_dreb`

    - The attribute `d_reb` can be obtained `d_oreb + d_dreb`.

    - The attribute `o_pts` can be obtained `2 * ( o_fgm + o_3pm ) + 3 * o_3pm + o_ftm`.

    - The attribute `d_pts` can be obtained `2 * ( d_fgm + d_3pm ) + 3 * d_3pm + d_ftm`.

- In relation to the attributes relating to team rebounding, the same ideas can be replicated for the opposing team's team rebounding.

    - The `tmTRB` attribute can be obtained `tmORB + tmDRB`.

    - The `opptmTRB` attribute can be obtained `opptmORB + opptmDRB`.

In [120]:
# eliminate redundant attributes

print(f"Before Teams had {len(tables['Teams'].columns)} columns")
tables['Teams'] = tables['Teams'].drop(['franchID','name','arena'], axis=1)
tables['Teams'] = tables['Teams'].drop(['GP','homeL','awayL','awayW'], axis=1)
tables['Teams'] = tables['Teams'].drop(['o_reb','d_reb','o_pts','d_pts'], axis=1)
print(f"After Teams has {len(tables['Teams'].columns)} columns")

Before Teams had 52 columns
After Teams has 41 columns


We start by looking at how many awards each player and coach received in a given year, and we add an `awards` column with that information to the `Coaches` and `Players Teams` datasets.

With that information, we can group it by teams, and add two new columns to the `Teams` table, `awards_coaches` and `awards_players`.

In [121]:
# join awards.csv in teams.csv

player_awards_count = tables['Awards Players'].groupby(['playerID', 'year']).size().reset_index(name='awards')
tables['Players Teams'] = tables['Players Teams'].merge(player_awards_count, on=['playerID', 'year'], how='left')
tables['Players Teams']['awards'] = tables['Players Teams']['awards'].fillna(0).astype(int)

coach_awards = tables['Awards Players'].rename(columns={'playerID': 'coachID'})
coach_awards_count = coach_awards.groupby(['coachID', 'year']).size().reset_index(name='awards')
tables['Coaches'] = tables['Coaches'] .merge(coach_awards_count, on=['coachID', 'year'], how='left')
tables['Coaches'] ['awards'] = tables['Coaches'] ['awards'].fillna(0).astype(int)

team_players_count = tables['Players Teams'].groupby(['tmID', 'year'])['awards'].sum().reset_index()
team_players_count = team_players_count.rename(columns={'awards': 'awards_players'})
tables['Teams'] = tables['Teams'].merge(team_players_count, on=[ 'year','tmID'], how='left')
tables['Teams']['awards_players'] = tables['Teams']['awards_players'].fillna(0).astype(int)

team_coaches_count = tables['Coaches'].groupby(['tmID', 'year'])['awards'].sum().reset_index()
team_coaches_count = team_coaches_count.rename(columns={'awards': 'awards_coaches'})
tables['Teams'] = tables['Teams'].merge(team_coaches_count, on=['tmID', 'year'], how='left')
tables['Teams']['awards_coaches'] = tables['Teams']['awards_coaches'].fillna(0).astype(int)

In [122]:
# join of teams_post.csv in teams.csv

tables['Teams'].columns = ['teams_' + col for col in tables['Teams'].columns]
tables['Teams Post'].columns = ['teams_post_' + col for col in tables['Teams Post'].columns]
tables['Coaches'].columns = ['coaches_' + col for col in tables['Coaches'].columns]

tables['Teams'] = pd.merge( tables['Teams'], tables['Teams Post'], left_on=['teams_tmID', 'teams_year'], right_on=['teams_post_tmID', 'teams_post_year'], how='left')
tables['Teams'] = tables['Teams'].drop( ['teams_post_year', 'teams_post_tmID', 'coaches_year'], axis=1, errors='ignore')

## 3. Prepare players_teams.csv

- In relation to the attributes referring to statistics of the games played, there are some attributes that are not necessary:

    - The attribute `rebounds` can be obtained `oRebounds + dRebounds`

    - The attribute `PostRebounds` can be obtained `PostoRebounds + PostdRebounds`

In [123]:
print(f"Before Players Teams had {len(tables['Players Teams'].columns)} columns")
tables['Players Teams'] = tables['Players Teams'].drop(['rebounds', 'PostRebounds'], axis=1)
print(f"After Players Teams has {len(tables['Players Teams'].columns)} columns")

Before Players Teams had 43 columns
After Players Teams has 41 columns


In [124]:
# join of players.csv in players_teams.csv

tables['Players Teams'].columns = ['players_teams_' + col for col in tables['Players Teams'].columns]
tables['Players'].columns = ['players_' + col for col in tables['Players'].columns]

tables['Players Teams'] = pd.merge( tables['Players Teams'], tables['Players'], left_on='players_teams_playerID', right_on='players_bioID', how='left')
tables['Players Teams'] = tables['Players Teams'].drop( ['players_bioID'], axis=1, errors='ignore')

In [125]:
# join of teams.csv in players_teams.csv

tables['Players Teams'] = pd.merge( tables['Players Teams'], tables['Teams'], left_on=['players_teams_year','players_teams_tmID'], right_on=['teams_year','teams_tmID'], how='left')
tables['Players Teams'] = tables['Players Teams'].drop( ['teams_year','teams_tmID'], axis=1, errors='ignore')

### 5 - Save prepared data

In [126]:
for table in tables.keys():
    table_to_csv(table)